In [1]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.cross_validation import train_test_split

/home/sergei/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/sergei/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
import numpy as np

In [3]:
def preprocess_image(img):
    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [4]:
image_gen_train = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #zca_whitening=True,
    rescale=1./255,
    rotation_range = 90,
    width_shift_range=.5,
    height_shift_range=.5,
    shear_range=90,
    zoom_range= [3, 3],
    #channel_shift_range: Float. Range for random channel shifts.
    horizontal_flip=True,
    vertical_flip= True,
    preprocessing_function = preprocess_image
)
image_gen_val = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #zca_whitening=True,
    rescale=1./255,
    preprocessing_function = preprocess_image
)

#training the image preprocessing
#image_gen.fit(x_train, augment=True)

In [5]:
train_generator = image_gen_train.flow_from_directory(
        './input/train_resized_224/train',
        target_size=(224, 224),
        batch_size=32)

validation_generator = image_gen_val.flow_from_directory(
        './input/train_resized_224/val',
        target_size=(224, 224),
        batch_size=32)

Found 974865 images belonging to 14951 classes.
Found 242443 images belonging to 14951 classes.


In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import ImageFile
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard

In [8]:
from keras.callbacks import EarlyStopping

es = EarlyStopping( 'val_loss', patience = 5, 
                        mode = 'min', min_delta = .1)

save_check = ModelCheckpoint('./model/model_step_mn_1.h5', monitor='val_loss',
                verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, 
                                 write_graph=True, write_grads=False, write_images=False, 
                                 embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)


In [6]:
# create the base pre-trained model
base_model = MobileNet(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(14951, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

/home/sergei/.local/lib/python3.6/site-packages/keras/applications/mobilenet.py:224: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [18]:
from gc import collect
collect()

49515

In [23]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

model.fit_generator(train_generator,
                       verbose = 1,
                         validation_data=validation_generator, 
                      callbacks=[save_check, es, tb],
                        steps_per_epoch = 500, epochs=30)
    
    # train the model on the new data for a few epochs
    #model.fit_generator(...)
# serialize model to JSON
model_json = model.to_json()
with open("./model/model_step_1_mb_end.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./model/model_step_1_mb_end.h5")
print("Saved model to disk")

Epoch 1/30
500/500 [==============================] - 1392s 3s/step - loss: 7.7871 - val_loss: 7.7950

Epoch 00001: val_loss improved from inf to 7.79503, saving model to ./model/model_step_mn_1.h5
Epoch 2/30
500/500 [==============================] - 1343s 3s/step - loss: 7.6843 - val_loss: 7.8813

Epoch 00002: val_loss did not improve from 7.79503
Epoch 3/30
500/500 [==============================] - 1348s 3s/step - loss: 7.6449 - val_loss: 7.8446

Epoch 00003: val_loss did not improve from 7.79503
Epoch 4/30
500/500 [==============================] - 1373s 3s/step - loss: 7.6922 - val_loss: 7.7976

Epoch 00004: val_loss did not improve from 7.79503
Epoch 5/30
500/500 [==============================] - 1362s 3s/step - loss: 7.6866 - val_loss: 7.8160

Epoch 00005: val_loss did not improve from 7.79503
Epoch 6/30
500/500 [==============================] - 1359s 3s/step - loss: 7.6910 - val_loss: 7.7931

Epoch 00006: val_loss improved from 7.79503 to 7.79306, saving model to ./model/mod

In [10]:
model.load_weights('./model/model_step_mn_1.h5')

In [11]:
save_check = ModelCheckpoint('./model/model_step_mn_2.h5', monitor='val_loss',
                verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

In [12]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2238773612059792538
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5482086400
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11358617874174306947
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [13]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [14]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_pad_1
6 conv_dw_1
7 conv_dw_1_bn
8 conv_dw_1_relu
9 conv_pw_1
10 conv_pw_1_bn
11 conv_pw_1_relu
12 conv_pad_2
13 conv_dw_2
14 conv_dw_2_bn
15 conv_dw_2_relu
16 conv_pw_2
17 conv_pw_2_bn
18 conv_pw_2_relu
19 conv_pad_3
20 conv_dw_3
21 conv_dw_3_bn
22 conv_dw_3_relu
23 conv_pw_3
24 conv_pw_3_bn
25 conv_pw_3_relu
26 conv_pad_4
27 conv_dw_4
28 conv_dw_4_bn
29 conv_dw_4_relu
30 conv_pw_4
31 conv_pw_4_bn
32 conv_pw_4_relu
33 conv_pad_5
34 conv_dw_5
35 conv_dw_5_bn
36 conv_dw_5_relu
37 conv_pw_5
38 conv_pw_5_bn
39 conv_pw_5_relu
40 conv_pad_6
41 conv_dw_6
42 conv_dw_6_bn
43 conv_dw_6_relu
44 conv_pw_6
45 conv_pw_6_bn
46 conv_pw_6_relu
47 conv_pad_7
48 conv_dw_7
49 conv_dw_7_bn
50 conv_dw_7_relu
51 conv_pw_7
52 conv_pw_7_bn
53 conv_pw_7_relu
54 conv_pad_8
55 conv_dw_8
56 conv_dw_8_bn
57 conv_dw_8_relu
58 conv_pw_8
59 conv_pw_8_bn
60 conv_pw_8_relu
61 conv_pad_9
62 conv_dw_9
63 conv_dw_9_bn
64 conv_dw_9_relu
65 conv_pw_9
66 conv_pw_9_

In [15]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:93]:
    layer.trainable = False
for layer in model.layers[93:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

model.fit_generator(train_generator,
                       verbose = 1,
                         validation_data=validation_generator, 
                      callbacks=[save_check, es, tb],
                        steps_per_epoch = 100, epochs=20)

Epoch 1/20
100/100 [==============================] - 16207s 162s/step - loss: 7.6001 - val_loss: 7.7776

Epoch 00001: val_loss improved from inf to 7.77758, saving model to ./model/model_step_mn_2.h5
Epoch 2/20
100/100 [==============================] - 1736s 17s/step - loss: 7.6444 - val_loss: 7.8022

Epoch 00002: val_loss did not improve from 7.77758
Epoch 3/20
100/100 [==============================] - 1033s 10s/step - loss: 7.5497 - val_loss: 7.7936

Epoch 00003: val_loss did not improve from 7.77758
Epoch 4/20
100/100 [==============================] - 997s 10s/step - loss: 7.5708 - val_loss: 7.8251

Epoch 00004: val_loss did not improve from 7.77758
Epoch 5/20
100/100 [==============================] - 993s 10s/step - loss: 7.5236 - val_loss: 7.7693

Epoch 00005: val_loss improved from 7.77758 to 7.76931, saving model to ./model/model_step_mn_2.h5
Epoch 6/20
100/100 [==============================] - 968s 10s/step - loss: 7.5944 - val_loss: 7.8041

Epoch 00006: val_loss did not 

In [7]:
model.load_weights('./model/model_step_mn_2.h5')

In [8]:
! unzip ./input/sample_submission.csv.zip

Archive:  ./input/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [9]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,landmarks
0,000088da12d664db,8815 0.03
1,0001623c6d808702,7249 0.61
2,0001bbb682d45002,5328 0.5
3,0002362830cfe3a3,4188 0.96
4,000270c9100de789,10506 0.82


In [37]:
import matplotlib.image as mpimg
import os

In [63]:
%%time
data = np.array([mpimg.imread('./input/test_resized_224/'+name) for name in os.listdir('./input/test_resized_224/')[:10]], dtype=np.float64)
print(data.shape)

(10, 224, 224, 3)
CPU times: user 29.3 ms, sys: 65.3 ms, total: 94.6 ms
Wall time: 92.9 ms


In [ ]:
%%time
for i in range(submission.shape[0]):
    if i % 10000 ==0:
        print('10000 loaded')
    try:
        pil_image = Image.open('./input/test_resized_224/'+submission.iloc[i, 0]+'.jpg')
        pil_image = preprocess_image(pil_image)*(1./255)
        y_prob = model.predict(pil_image, batch_size=1)
        submission.iloc[i, 1] = y_prob.argmax(axis=-1)[0]
    except:
        continue

10000 loaded
10000 loaded
10000 loaded


In [ ]:
submission.head(10)

In [ ]:
submission.head(1000).landmarks.nunique()